In [29]:
import pandas as pd

def nyc_etl_pipeline(input_file, output_file):
        
    df = pd.read_parquet(input_file)
    
   
    df["passenger_count"]      = df["passenger_count"].fillna(1)
    df["RatecodeID"]           = df["RatecodeID"].fillna(1)
    df["store_and_fwd_flag"]   = df["store_and_fwd_flag"].fillna("N")
    df["congestion_surcharge"] = df["congestion_surcharge"].fillna(0)
    df["Airport_fee"]          = df["Airport_fee"].fillna(0)

   
    df.loc[df["congestion_surcharge"] < 0, "congestion_surcharge"] = 0
    df.loc[df["Airport_fee"] < 0, "Airport_fee"] = 0

   
    df["tpep_pickup_datetime"]  = pd.to_datetime(df["tpep_pickup_datetime"])
    df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])
    df["trip_duration_min"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60

  
    df = df[df["trip_distance"] > 0]
    df = df[df["trip_distance"] <= 100]
    df = df[df["fare_amount"] >= 0]
    df = df[df["fare_amount"] <= 500]
    df = df[df["trip_duration_min"] > 0]
    df = df[df["trip_duration_min"] <= 360]
    
    
    df.to_parquet(output_file, index=False)
    print("Clean dataset after ETL saved as:output_file")
    
    return df

In [30]:
df_final_dataset = nyc_etl_pipeline("yellow_tripdata_2025-01.parquet", "yellow_tripdata_2025-01_storedaftercleaning.parquet")
print(df_final_dataset.shape)

Clean dataset after ETL saved as:output_file
(3251980, 21)
